In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"] = "/home/ec2-user/spark-2.4.4-bin-hadoop2.7/python"
os.environ["JAVA_HOME"] = "/usr/java/jdk1.8.0_161/jre"
os.environ["SPARK_HOME"] = "/home/ec2-user/spark-2.4.4-bin-hadoop2.7"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] + "/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] + "/pyspark.zip")

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("Trial").getOrCreate()

22/04/25 19:29:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
data = spark.read.csv("vgsales.csv", header = True, inferSchema= True)

In [5]:
data.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- NA_Sales: double (nullable = true)
 |-- EU_Sales: double (nullable = true)
 |-- JP_Sales: double (nullable = true)
 |-- Other_Sales: double (nullable = true)
 |-- Global_Sales: double (nullable = true)



In [6]:
df = data.createOrReplaceTempView("sales")

### Question 

Does the list contain the game 'God of War'

In [7]:
spark.sql("SELECT * FROM sales WHERE Name == 'God of War' ").show()

+----+----------+--------+----+------+--------------------+--------+--------+--------+-----------+------------+
|Rank|      Name|Platform|Year| Genre|           Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|
+----+----------+--------+----+------+--------------------+--------+--------+--------+-----------+------------+
| 246|God of War|     PS2|2005|Action|Sony Computer Ent...|    2.71|    1.29|    0.02|       0.43|        4.45|
+----+----------+--------+----+------+--------------------+--------+--------+--------+-----------+------------+



### Question 

Which market is the best to sell video games?

ANS: Noth America

In [8]:
spark.sql("SELECT SUM(NA_Sales),SUM(EU_Sales),SUM(JP_Sales),SUM(Other_Sales) FROM sales").show()

+-----------------+----------------+------------------+-----------------+
|    sum(NA_Sales)|   sum(EU_Sales)|     sum(JP_Sales)| sum(Other_Sales)|
+-----------------+----------------+------------------+-----------------+
|4392.950000000332|2434.13000000055|1291.0199999999018|797.7499999998826|
+-----------------+----------------+------------------+-----------------+



### Questio

How many games have global sales more than 10 million dollars?

ANS: 62

In [9]:
spark.sql("SELECT * FROM sales WHERE Global_Sales> 10").count()

62

In [10]:
spark.sql("SELECT COUNT (Global_Sales) FROM sales WHERE Global_Sales>10").show()

+-------------------+
|count(Global_Sales)|
+-------------------+
|                 62|
+-------------------+



### Question 
Using a an SQL query findout which Role-Playing game has third highest sales in Japan.

ANS = 'Pokemon Diamond/Pokemon Pearl'

In [11]:
RPG = spark.sql("SELECT * FROM sales WHERE Genre == 'Role-Playing' ORDER BY JP_Sales DESC ").head(3)[2].asDict()

In [12]:
RPG['Name']

'Pokemon Diamond/Pokemon Pearl'

### Question 

Which the most popular game Genre in Europe? 

ANS: 'Action'  

In [13]:
PG = spark.sql("SELECT Genre, SUM(EU_Sales) FROM sales  GROUP BY Genre ORDER BY SUM(EU_Sales) DESC ").head(1)[0].asDict()

In [14]:
PG['Genre']

'Action'